## Step 1: Import Packages

In [1]:
import re
import pandas as pd

## Step 2: Import Dataset and Perform Initial Preprocessing

In [2]:
# Import data from Excel to Pandas dataframe. Make sure to include PatientICN/Patientsid, PFT date, and ReportText columns
df = pd.read_excel('[Insert Directory Here]/[Insert File Name Here].xlsx')

# Convert ReportText column to string and remove carriage returns and extra spaces between words
df['ReportText'] = df['ReportText'].astype('str')
df['ReportText'] = df['ReportText'].str.replace(r'\s+', ' ',regex=True).replace(r'\n+', ' ', regex=True)

## Step 3: Create Snippet Extraction Function after identifying template snippets with the following code:
```
n = 1
for index,row in df.sample(frac=1)[:100].iterrows():
    print(f"Row Number: {n}")
    print(f"Note Content: {row['ReportText']}")
    print(f"PFT Date: {row['pft_date']}")
    print('-'*100)
    n+=1
```

In [4]:
# Function to create snippet based on template start phrase(s)
def extract_pft_context(text):
    pattern = re.compile(r'Spirometry:.{0,500}|Spirometry Report.{0,1000}', re.IGNORECASE)
    matches = pattern.findall(text)
    return ''.join(matches)

## Step 4: Generate `Snippet` column by applying above function to the `ReportText` column

In [ ]:
# Create a copy of the df to avoid warnings and run the ReportText column through the function to create the Snippet column
df = df.copy(deep = True)
df['Snippet'] = df['ReportText'].apply(extract_pft_context)

# Create new dataframe where all rows with no snippet are dropped
notes_with_fev = df[df['Snippet'] != ''].reset_index(drop=True)

##### You can validate snippets with the following code:
```
n = 1
for index,row in notes_with_fev.sample(frac=1)[:100].iterrows():
    print(f"Row Number: {n}")
    print(row['ReportText'])
    print(row['pft_date'])
    print('-'*100)
    n+=1
```

## Step 5: Initialize PFT Classification Function

In [9]:
def classify_pft(row):
    fev1_abs_pre = []
    fev1_abs_post = []
    fev1_fvc_pre = []
    fev1_fvc_post = []
    fev1_perc_pred_pre = []
    fev1_perc_pred_post = []
    
    text = row['Snippet']
    
    # FEV1 abs pre 
    fev1_abs_pre_pattern = re.compile(r'Pre/Best.*?FEV1.*?\d*\.\d+\s\d*\d.\d+\s(\d*\.\d+)', re.IGNORECASE)
    fev1_abs_pre_pattern_results = fev1_abs_pre_pattern.findall(text)
    
    if fev1_abs_pre_pattern_results:
        fev1_abs_pre.append(fev1_abs_pre_pattern_results)
        
    # FEV1 abs post 
    fev1_abs_post_pattern = re.compile(r'Pre/Best.*?FEV1.*?\d*\.\d+\s\d*\d.\d+\s\d*\.\d+\s\d{2}\s(\d*\.\d+)', re.IGNORECASE)
    fev1_abs_post_pattern_results = fev1_abs_post_pattern.findall(text)
    
    if fev1_abs_post_pattern_results:
        fev1_abs_post.append(fev1_abs_post_pattern_results)
        
    # FEV1 abs pre 2 
    fev1_abs_pre_pattern_2 = re.compile(r'Pre/Base.*?FEV1.*?(\d*\.\d+)', re.IGNORECASE)
    fev1_abs_pre_pattern_2_results = fev1_abs_pre_pattern_2.findall(text)
    
    if fev1_abs_pre_pattern_2_results:
        fev1_abs_pre.append(fev1_abs_pre_pattern_2_results)
        
    # FEV1 abs post 2 
    fev1_abs_post_pattern_2 = re.compile(r'Pre/Base.*?FEV1.*?\d*\.\d+\s\d{2}\s\d*\.\d+\s(\d*\.\d+)', re.IGNORECASE)
    fev1_abs_post_pattern_2_results = fev1_abs_post_pattern_2.findall(text)
    
    if fev1_abs_post_pattern_2_results:
        fev1_abs_post.append(fev1_abs_post_pattern_2_results)
    
    # FEV1/FVC pre 1 
    fev1_fvc_pre_pattern = re.compile(r'Pre/Best.*?FEV1/FVC.*?\d*\.\d+\s\d*\d.\d+\s(\d*\.\d+)', re.IGNORECASE)
    fev1_fvc_pre_pattern_results = fev1_fvc_pre_pattern.findall(text)
    
    if fev1_fvc_pre_pattern_results:
        fev1_fvc_pre.append(fev1_fvc_pre_pattern_results)
        
    # FEV1/FVC post 1 
    fev1_fvc_post_pattern = re.compile(r'Pre/Best.*?FEV1/FVC.*?\d*\.\d+\s\d*\d.\d+\s\d*\.\d+\s\d{2,3}\s(\d*\.\d+)', re.IGNORECASE)
    fev1_fvc_post_pattern_results = fev1_fvc_post_pattern.findall(text)
    
    if fev1_fvc_post_pattern_results:
        fev1_fvc_post.append(fev1_fvc_post_pattern_results)
        
    # FEV1/FVC pre 2 
    fev1_fvc_pre_2_pattern = re.compile(r'Pre/Base.*?FEV1/FVC.*?(\d{2,3})', re.IGNORECASE)
    fev1_fvc_pre_2_pattern_results = fev1_fvc_pre_2_pattern.findall(text)
    
    if fev1_fvc_pre_2_pattern_results:
        fev1_fvc_pre.append(fev1_fvc_pre_2_pattern_results)
        
    # FEV1/FVC post 2 
    fev1_fvc_post_2_pattern = re.compile(r'Pre/Base.*?FEV1/FVC.*?\d{2,3}\s\d{2,3}\s\d{2,3}\s(\d{2,3})', re.IGNORECASE)
    fev1_fvc_post_2_pattern_results = fev1_fvc_post_2_pattern.findall(text)
    
    if fev1_fvc_post_2_pattern_results:
        fev1_fvc_post.append(fev1_fvc_post_2_pattern_results)
        
    # FEV1 perc pred pre 1 
    fev1_perc_pred_pre_1_pattern = re.compile(r'Pre/Best.*?FEV1.*?\d*\.\d+\s\d*\d.\d+\s\d*\.\d+\s(\d{2})', re.IGNORECASE)
    fev1_perc_pred_pre_1_pattern_results = fev1_perc_pred_pre_1_pattern.findall(text)
    
    if fev1_perc_pred_pre_1_pattern_results:
        fev1_perc_pred_pre.append(fev1_perc_pred_pre_1_pattern_results)
        
    # FEV1 perc pred post 1 
    fev1_perc_pred_post_1_pattern = re.compile(r'Pre/Best.*?FEV1.*?\d*\.\d+\s\d*\d.\d+\s\d*\.\d+\s\d{2}\s\d*\.\d+\s(\d{2})', re.IGNORECASE)
    fev1_perc_pred_post_1_pattern_results = fev1_perc_pred_post_1_pattern.findall(text)
    
    if fev1_perc_pred_post_1_pattern_results:
        fev1_perc_pred_post.append(fev1_perc_pred_post_1_pattern_results)
        
    # FEV1 percent pred pre 2 
    fev1_perc_pred_pre_2_pattern = re.compile(r'Pre/Base.*?FEV1.*?\d*\.\d+\s(\d{2})', re.IGNORECASE)
    fev1_perc_pred_pre_2_pattern_results = fev1_perc_pred_pre_2_pattern.findall(text)
    
    if fev1_perc_pred_pre_2_pattern_results:
        fev1_perc_pred_pre.append(fev1_perc_pred_pre_2_pattern_results)
        
    # FEV1 percent pred post 2 
    fev1_perc_pred_post_2_pattern = re.compile(r'Pre/Base.*?FEV1.*?\d*\.\d+\s\d{2}\s\d*\.\d+\s\d*\.\d+\s(\d{2})', re.IGNORECASE)
    fev1_perc_pred_post_2_pattern_results = fev1_perc_pred_post_2_pattern.findall(text)
    
    if fev1_perc_pred_post_2_pattern_results:
        fev1_perc_pred_post.append(fev1_perc_pred_post_2_pattern_results)
        
    # FEV1 qualitative high
    fev1_qual_high = []
    
    fev1_qual_high_pattern = re.compile(r'(spirometry: is normal)', re.IGNORECASE)
    fev1_qual_high_pattern_result = fev1_qual_high_pattern.findall(text)
    
    if fev1_qual_high_pattern_result:
        fev1_qual_high.append(fev1_qual_high_pattern_result)
        
    # FEV1 qualitative low
    fev1_qual_low = []
    
    fev1_qual_low_pattern = re.compile(r'(mild obstructive ventilatory defect|moderate obstructive ventilatory defect|moderately severe obstructive ventilatory defect|moderate-severe obstructive ventilatory defect|severe obstructive ventilatory defect|very severe obstructive ventilatory defect)', re.IGNORECASE)
    fev1_qual_low_pattern_result = fev1_qual_low_pattern.findall(text)
    
    if fev1_qual_low_pattern_result:
        fev1_qual_low.append(fev1_qual_low_pattern_result)
        
    return pd.Series({'FEV1_Abs_Pre': fev1_abs_pre if fev1_abs_pre else None,
                     'FEV1_Abs_Post': fev1_abs_post if fev1_abs_post else None,
                      'FEV1_FVC_Pre': fev1_fvc_pre if fev1_fvc_pre else None,
                      'FEV1_FVC_Post': fev1_fvc_post if fev1_fvc_post else None,
                     'FEV1_Perc_Pred_Pre': fev1_perc_pred_pre if fev1_perc_pred_pre else None,
                     'FEV1_Perc_Pred_Post': fev1_perc_pred_post if fev1_perc_pred_post else None,
                     'FEV1_Qual_High': fev1_qual_high if fev1_qual_high else None,
                     'FEV1_Qual_Low': fev1_qual_low if fev1_qual_low else None})

## Step 6: Run dataframe through the PFT extraction function

In [10]:
results = notes_with_pft.join(notes_with_pft.apply(classify_pft, axis = 1))

### Fix date column

In [11]:
results['pft_date'] = results['pft_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

## Step 7: Extract values from FEV1 % predicted, FEV1:FVC pre-BD variables, and qualitative data

In [12]:
def extract_perc_pred_value(nested_list):
    if nested_list is not None:
        return int(nested_list[0][0])
    
# Create new variable 'FEV1_Perc_Pred' to hold extracted FEV1 % predicted quantitative value
results['FEV1_Perc_Pred'] = results['FEV1_Perc_Pred_Pre'].apply(extract_perc_pred_value)

def extract_fev1_fevc_value(nested_list):
    if nested_list is not None:
        return float(nested_list[0][0])

# Create new variable 'fev1_fvc' to hold FEV1:FVC quantitative value
results['fev1_fvc'] = results['FEV1_FVC_Pre'].apply(extract_fev1_fevc_value)

def extract_fev1_qualitative(nested_list):
    if nested_list is not None:
        return str(nested_list[0][0])

# Create new variables to hold qualitative data
results['fev1_qual_neg'] = results['FEV1_Qual_Low'].apply(extract_fev1_qualitative)
results['fev1_qual_pos'] = results['FEV1_Qual_High'].apply(extract_fev1_qualitative)

## Step 8: Create mapping functions to map quantitative obstruction results to the correct classification

In [13]:
def fev1_severity(value):
    if value >= 80:
        return "Normal"
    if 70 <= value <= 79:
        return "Mild"
    if 60 <= value <= 69:
        return "Moderate"
    if 50 <= value <= 59:
        return "Moderately Severe"
    if 35 <= value < 50:
        return "Severe"
    if value < 35:
        return "Very Severe"

def obstruction(value):
    if value >= 70:
        return "Normal"
    if value < 70:
        return "Reduced"
    
# Create new variables 'FEV1_Severity' and 'Obstruction', which hold the mapped values
results['FEV1_Severity'] = results['FEV1_Perc_Pred'].map(fev1_severity)
results['Obstruction'] = results['fev1_fvc'].map(obstruction)

## Step 9a: Map qualitative values to FEV1 Severity if note did not contain extractable quantitative data

In [14]:
def fev1_severity_from_qual(row):
    if row['FEV1_Severity'] is None and row['fev1_qual_pos'] == 'SPIROMETRY: Is normal':
        return "Normal"
    if row['FEV1_Severity'] is None and row['fev1_qual_neg'] == 'mild obstructive ventilatory defect':
        return "Mild"
    elif row['FEV1_Severity'] is None and row['fev1_qual_neg'] == 'moderate obstructive ventilatory defect':
        return "Moderate"
    elif row['FEV1_Severity'] is None and row['fev1_qual_neg'] in ['moderately severe obstructive ventilatory defect', 'moderate-severe obstructive ventilatory defect']:
        return "Moderately Severe"
    elif row['FEV1_Severity'] is None and row['fev1_qual_neg'] == 'severe obstructive ventilatory defect':
        return "Severe"
    elif row['FEV1_Severity'] is None and row['fev1_qual_neg'] == 'very severe obstructive ventilatory defect':
        return "Very Severe"
    else:
        return row['FEV1_Severity']
    
# If quantitative data is missing for FEV1 % predicted, use available qualitative data to map value
results['FEV1_Severity'] = results.apply(fev1_severity_from_qual, axis = 1)     

## Step 9b: Map qualitative values to Obstruction if note did not contain extractable quantitative data

In [15]:
def obstruction_from_qual(row):
    if row['Obstruction'] is None and row['fev1_qual_pos'] == 'SPIROMETRY: Is normal':
        return "Normal"
    elif row['Obstruction'] is None and row['fev1_qual_neg'] is not None:
        return "Reduced"
    else:
        return row['Obstruction']

# If qunatitative data is missing for Obstruction, use available qualitative data to map value
results['Obstruction'] = results.apply(obstruction_from_qual, axis = 1)

## Step 10: Drop duplicate rows and rows missing any extracted PFT data

In [16]:
# List columns to define on which variables to drop duplicates
list_cols = ['Obstruction', 'FEV1_Severity', 'FEV1_Abs_Pre', 'FEV1_Perc_Pred_Pre', 'FEV1_FVC_Pre', 'FEV1_Abs_Post', 'FEV1_Perc_Pred_Post', 'FEV1_FVC_Post', 'FEV1_Qual_High', 'FEV1_Qual_Low']

# Drop rows that have no extracted PFT values
results = results.dropna(subset = list_cols, how='all')

# Convert data types of columns in list_cols to string
for col in list_cols:
    results[col] = results[col].apply(lambda x: str(x))

# Drop duplicates of PFT results based on columns of interest + PatientID and PFT date
results = results.drop_duplicates(subset=['PatientICN', 'pft_date', 'Obstruction', 'FEV1_Severity', 'FEV1_Abs_Pre', 'FEV1_Perc_Pred_Pre', 'FEV1_FVC_Pre', 'FEV1_Abs_Post', 'FEV1_Perc_Pred_Post', 'FEV1_FVC_Post', 'FEV1_Qual_High', 'FEV1_Qual_Low'])

# Replace cells with 'None' values to empty string for ease of readability in the output Excel file
results.replace('None','',inplace=True)

4134
3498
3345


## Step 11: Collapse notes from same PFT with multiple notes containing values for different variables

In [17]:
# Define columns that will keep the max value if the two rows being merged have different values for.
columns_to_max = ['PatientSID', 'Obstruction', 'FEV1_Severity', 'FEV1_Abs_Pre', 'FEV1_Perc_Pred_Pre', 'FEV1_FVC_Pre', 'FEV1_Abs_Post', 'FEV1_Perc_Pred_Post', 'FEV1_FVC_Post', 'FEV1_Qual_High', 'FEV1_Qual_Low']

# This function ensures that we don't lose one of the snippets upon the merge butu rather append them together.
def concatenate_strings(series):
    return ''.join(series.unique())

# Define aggregation function to keep the  max value for columns that have different data in both rows
agg_funcs = {col: 'max' for col in columns_to_max}

# Create concatenated snippets for merged rows (instead of taking the "max" snippet value)
agg_funcs['Snippet'] = concatenate_strings

# Regenerate dataframe with the collapsed rows for identical PFTs with multiple notes
results = results.groupby(['PatientICN','pft_date'], sort = False).agg(agg_funcs).reset_index()

## Step 12: Export dataframe to Excel

In [20]:
# Select columns to export
columns_to_export = ['Snippet', 'PatientICN', 'PatientSID', 'pft_date', 'Obstruction', 'FEV1_Severity', 'FEV1_Abs_Pre', 'FEV1_Perc_Pred_Pre', 'FEV1_FVC_Pre', 'FEV1_Abs_Post', 'FEV1_Perc_Pred_Post', 'FEV1_FVC_Post', 'FEV1_Qual_High', 'FEV1_Qual_Low']

# Define desired output directory, file name, and file path
output_dir = '[Insert Your Directory Here]/'
file_name = '[Insert Your Filename Here].xlsx'
full_path = output_dir + file_name

# Export data as .xslx file
results.to_excel(full_path, columns = columns_to_export, index=False)